<a href="https://colab.research.google.com/github/vsnupoudel/data-science-capstone-JohnsHopkins/blob/master/Embedding_Prediction_instead_for_Next_word_Prediction_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the text files and the Embedding file

In [4]:
!gdown --id 1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE -O '/content/en_US.zip'

Downloading...
From: https://drive.google.com/uc?id=1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE
To: /content/en_US.zip
262MB [00:08, 30.8MB/s]


In [5]:
!unzip -o en_US.zip 

Archive:  en_US.zip
  inflating: en_US/en_US.blogs.txt   
  inflating: en_US/en_US.news.txt    
  inflating: en_US/en_US.twitter.txt  


In [6]:
!gdown --id 1r3MHZdvnlC_BG8z6Jr1td-zJTC2GdqYK -O 'glove.6B.50d.txt' 

Downloading...
From: https://drive.google.com/uc?id=1r3MHZdvnlC_BG8z6Jr1td-zJTC2GdqYK
To: /content/glove.6B.50d.txt
171MB [00:01, 130MB/s] 


In [7]:
# # establish connections with three files 
with open('./en_US/en_US.blogs.txt','r') as f:
  blogs= f.readlines(199999)

References :


 [Towards Data Science Article](https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f)

[Machine Learning Mastery](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)

In [8]:
string = ' '.join(blogs) 
del blogs

In [9]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
# from keras.utils import to_categorical

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Removing non word characters

In [11]:
string = re.sub(pattern='\W+', repl=" ", string = string ).lower()

Tokenize and generate ngrams

In [12]:
tokens = word_tokenize(string)
train_len = 3
text_sequences = []

for i in range(train_len,len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)

In [13]:
text_sequences[0:5]

[['in', 'the', 'years'],
 ['the', 'years', 'thereafter'],
 ['years', 'thereafter', 'most'],
 ['thereafter', 'most', 'of'],
 ['most', 'of', 'the']]

Read in 50 dimensional embeddings

In [14]:
import pandas as pd
import numpy as np
colnames = ['word']+[ 'dim_'+str(i) for i in range(1,51)]
Embed50 = pd.read_csv('glove.6B.50d.txt', engine='python'
                  ,header=None, delim_whitespace=True ,
                  names = colnames)
Embed50.head(2)

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.24968,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.65660,0.27843,-0.147670,-0.55677,0.14658,-0.00951,0.011658,0.10204,-0.127920,-0.84430,-0.12181,-0.016801,-0.33279,-0.15520,-0.23131,-0.191810,-1.8823,-0.76746,0.099051,-0.421250,-0.19526,4.0071,-0.18594,-0.522870,-0.31681,0.000592,0.007445,0.17778,-0.15897,0.012041,-0.054223,-0.298710,-0.15749,-0.34758,-0.045637,-0.44251,0.18785,0.002785,-0.18411,-0.115140,-0.78581
1,",",0.013441,0.23682,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.23938,0.13001,-0.063734,-0.39575,-0.48162,0.23291,0.090201,-0.13324,0.078639,-0.41634,-0.15428,0.100680,0.48891,0.31226,-0.12520,-0.037512,-1.5179,0.12612,-0.024420,-0.042961,-0.28351,3.5416,-0.11956,-0.014533,-0.14990,0.218640,-0.334120,-0.13872,0.31806,0.703580,0.448580,-0.080262,0.63003,0.32111,-0.467650,0.22786,0.36034,-0.378180,-0.56657,0.044691,0.30392


Tokenizer class from keras.preprocessing.text

In [15]:
# del tokens
# del seq
# del string

Delete those rows which have words, that are not there in the pretrained embeddings. Found 100 such sequences

In [16]:
embedlist = list(Embed50.word)
delete_list = []
for seq in text_sequences:
  check =  all(item in embedlist for item in seq)
  if not check:
    delete_list.append(seq)

text_sequences_del = [x for x in text_sequences if x not in delete_list]
del text_sequences

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences_del)
sequences = tokenizer.texts_to_sequences(text_sequences_del)
#vocabulary size increased by 1 
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.asarray ( sequences , dtype='int32')

In [18]:
dic = pd.DataFrame( tokenizer.word_index.items(), columns= ['word','index'])

In [19]:
dic.head(2)

,word,index
0,the,1
1,to,2


In [32]:
Embed50.head(2)

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.24968,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.65660,0.27843,-0.147670,-0.55677,0.14658,-0.00951,0.011658,0.10204,-0.127920,-0.84430,-0.12181,-0.016801,-0.33279,-0.15520,-0.23131,-0.191810,-1.8823,-0.76746,0.099051,-0.421250,-0.19526,4.0071,-0.18594,-0.522870,-0.31681,0.000592,0.007445,0.17778,-0.15897,0.012041,-0.054223,-0.298710,-0.15749,-0.34758,-0.045637,-0.44251,0.18785,0.002785,-0.18411,-0.115140,-0.78581
1,",",0.013441,0.23682,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.23938,0.13001,-0.063734,-0.39575,-0.48162,0.23291,0.090201,-0.13324,0.078639,-0.41634,-0.15428,0.100680,0.48891,0.31226,-0.12520,-0.037512,-1.5179,0.12612,-0.024420,-0.042961,-0.28351,3.5416,-0.11956,-0.014533,-0.14990,0.218640,-0.334120,-0.13872,0.31806,0.703580,0.448580,-0.080262,0.63003,0.32111,-0.467650,0.22786,0.36034,-0.378180,-0.56657,0.044691,0.30392


In [ ]:
word_embeds_pre = pd.merge(
    left = dic,
    right = Embed50 ,
    how="inner",
    on="word" ).iloc[: , 2:52] 

In [21]:
word_embeds_pre.shape

(6957, 50)

In [22]:
word_embeds_pre.head(2)

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,0.41800,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.656600,0.27843,-0.14767,-0.55677,0.14658,-0.00951,0.011658,0.102040,-0.12792,-0.84430,-0.12181,-0.016801,-0.33279,-0.15520,-0.23131,-0.191810,-1.8823,-0.76746,0.099051,-0.42125,-0.19526,4.0071,-0.18594,-0.52287,-0.31681,0.000592,0.007445,0.17778,-0.158970,0.012041,-0.054223,-0.298710,-0.157490,-0.34758,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
1,0.68047,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,-0.085253,0.17118,0.22419,-0.10046,-0.43653,0.33418,0.678460,0.057204,-0.34448,-0.42785,-0.43275,0.559630,0.10032,0.18677,-0.26854,0.037334,-2.0932,0.22171,-0.398680,0.20912,-0.55725,3.8826,0.47466,-0.95658,-0.37788,0.208690,-0.327520,0.12751,0.088359,0.163510,-0.216340,-0.094375,0.018324,0.21048,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


In [23]:
# X and Y
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
# train_targets = to_categorical(  train_targets 
#                               , num_classes = vocabulary_size  )
seq_len = train_inputs.shape[1]

In [24]:
train_inputs.shape

(36584, 2)

In [25]:
train_targets.shape

(36584,)

In [46]:
y_list = list( np.asarray( text_sequences_del)[: , -1])
ydf = pd.DataFrame( data = y_list, columns = ['word'])
ydf.head(2)

,word
0,years
1,thereafter


In [49]:
outcome = pd.merge(
    left = ydf,
    right = Embed50 ,
    how="left",
    on='word' )

train_targets = outcome.iloc[: , 1:52] 

In [50]:
train_targets.head(2)

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,0.169620,0.434400,-0.042106,-0.63324,-0.12780,0.53668,-1.06620,-0.32629,-0.50079,0.10247,-0.021968,-0.35105,-0.641530,-0.42454,1.38360,-0.13543,-0.24754,0.22156,-0.65563,0.44424,0.17017,0.35816,0.56379,-0.480440,-0.14765,-1.62900,-0.31308,-0.47217,0.02659,0.47603,3.4619,0.12069,-0.045344,-0.47303,0.285690,-0.077584,-0.16447,0.71810,0.26170,-0.16841,-1.24500,-0.076188,0.17493,0.24507,-0.63801,-0.21096,-0.499180,-0.50108,-0.77040,-0.322340
1,-0.083242,-0.023838,-0.083964,-0.64821,-0.26937,-0.24057,-0.53924,0.73386,-0.60762,-0.59791,0.478670,0.28169,0.086011,-0.42519,0.46724,0.22510,-1.22500,-0.28048,-0.36225,0.40243,0.80369,-0.21699,0.83494,0.008353,0.37519,-0.61667,0.24404,-0.64431,0.20273,0.42442,1.8757,-0.30970,-0.012824,-0.64928,0.081936,-0.031222,0.27935,0.39467,-0.25122,0.10870,-0.30874,-0.601560,-0.34765,-0.11363,-0.63938,-0.28650,0.017039,-0.86755,-0.32626,0.043695


In [51]:
# Define model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
model = Sequential()
embed_layer = model.add(Embedding( input_dim = vocabulary_size-1
                                  , output_dim = 50 
                                  , input_length = seq_len
                                  , weights = [word_embeds_pre]
                                  , trainable = False
                                   ) )
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(100,activation='relu'))
model.add(Dense(50))

# compiling the network
model.compile(loss='mse', optimizer='adam'
, metrics=['mse', 'mae'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 50)             347850    
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 100)            60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
Total params: 448,600
Trainable params: 100,750
Non-trainable params: 347,850
_________________________________________________________________


In [53]:
#fit model
model.fit(train_inputs,train_targets,epochs=55,verbose=1)

Epoch 1/55
1144/1144 [==============================] - 27s 5ms/step - loss: 0.2768 - mse: 0.2768 - mae: 0.3881
Epoch 2/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2520 - mse: 0.2520 - mae: 0.3767
Epoch 3/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2495 - mse: 0.2495 - mae: 0.3751
Epoch 4/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2488 - mse: 0.2488 - mae: 0.3743
Epoch 5/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2465 - mse: 0.2465 - mae: 0.3728
Epoch 6/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2461 - mse: 0.2461 - mae: 0.3728
Epoch 7/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2434 - mse: 0.2434 - mae: 0.3707
Epoch 8/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2414 - mse: 0.2414 - mae: 0.3690
Epoch 9/55
1144/1144 [==============================] - 5s 5ms/step - loss: 0.2388 - mse: 0.2388 - mae: 0.3670


Predicting:

In [56]:
train_inputs[1, : ]

array([  1, 143], dtype=int32)

In [59]:
text_sequences_del[1]

['the', 'years', 'thereafter']

In [64]:
# model.predict( )
ypred = model.predict( np.expand_dims( np.array(train_inputs[1, : ]) , axis=0 )  )
ypred

array([[ 0.41172302, -0.00390446,  0.0364302 , -0.18941557,  0.41671234,
         0.11135185, -0.45541477,  0.00719416, -0.24578191, -0.02588872,
         0.01855587,  0.16312088, -0.3977009 , -0.16054949,  0.5621468 ,
         0.33557665,  0.14575917, -0.09494512, -0.27842757, -0.47066993,
        -0.02791364,  0.21501324,  0.31980553, -0.04331503,  0.23916422,
        -1.7652749 , -0.333898  ,  0.0845996 ,  0.313751  , -0.32377082,
         3.3916001 ,  0.22788896, -0.32822946, -0.34605497,  0.02683558,
        -0.13865562,  0.08889303,  0.17132705,  0.03008823, -0.16776395,
        -0.22602758,  0.13868384,  0.04337002,  0.17825586, -0.04871222,
         0.06845473, -0.23917818, -0.06586529, -0.1051907 ,  0.0135147 ]],
      dtype=float32)

Calculate distance with Embed50

In [60]:
Embed50.head(2)

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.24968,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.65660,0.27843,-0.147670,-0.55677,0.14658,-0.00951,0.011658,0.10204,-0.127920,-0.84430,-0.12181,-0.016801,-0.33279,-0.15520,-0.23131,-0.191810,-1.8823,-0.76746,0.099051,-0.421250,-0.19526,4.0071,-0.18594,-0.522870,-0.31681,0.000592,0.007445,0.17778,-0.15897,0.012041,-0.054223,-0.298710,-0.15749,-0.34758,-0.045637,-0.44251,0.18785,0.002785,-0.18411,-0.115140,-0.78581
1,",",0.013441,0.23682,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.23938,0.13001,-0.063734,-0.39575,-0.48162,0.23291,0.090201,-0.13324,0.078639,-0.41634,-0.15428,0.100680,0.48891,0.31226,-0.12520,-0.037512,-1.5179,0.12612,-0.024420,-0.042961,-0.28351,3.5416,-0.11956,-0.014533,-0.14990,0.218640,-0.334120,-0.13872,0.31806,0.703580,0.448580,-0.080262,0.63003,0.32111,-0.467650,0.22786,0.36034,-0.378180,-0.56657,0.044691,0.30392


In [71]:
  from scipy.spatial import distance
def distance(p1):
  return distance.euclidean(p1, ypred)


In [101]:
type( Embed50.iloc[:, 1:51].values)

numpy.ndarray

In [103]:
import time
start_time = time.time()

dist = map( lambda x : distance(x) , Embed50.iloc[:, 1:51].values )

end_time = time.time()
print( 'time taken: ', end_time - start_time)

time taken:  0.04667806625366211


In [104]:
listdist = list(dist)

In [107]:
distarray = np.asarray(listdist)

In [105]:
len(listdist)

400000

In [106]:
Embed50.shape

(400000, 51)

In [109]:
np.argmin(distarray)

34

In [113]:
Embed50.head(40)

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,dim_40,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.249680,-0.412420,0.121700,0.345270,-0.044457,-0.496880,-0.178620,-0.000660,-0.656600,0.278430,-0.147670,-0.556770,0.146580,-0.009510,0.011658,0.102040,-0.127920,-0.844300,-0.121810,-0.016801,-0.332790,-0.155200,-0.231310,-0.191810,-1.88230,-0.767460,0.099051,-0.421250,-0.195260,4.0071,-0.185940,-0.522870,-0.316810,0.000592,0.007445,0.177780,-0.158970,0.012041,-0.054223,-0.298710,-0.157490,-0.347580,-0.045637,-0.442510,0.187850,0.002785,-0.184110,-0.115140,-0.785810
1,",",0.013441,0.236820,-0.168990,0.409510,0.638120,0.477090,-0.428520,-0.556410,-0.364000,-0.239380,0.130010,-0.063734,-0.395750,-0.481620,0.232910,0.090201,-0.133240,0.078639,-0.416340,-0.154280,0.100680,0.488910,0.312260,-0.125200,-0.037512,-1.51790,0.126120,-0.024420,-0.042961,-0.283510,3.5416,-0.119560,-0.014533,-0.149900,0.218640,-0.334120,-0.138720,0.318060,0.703580,0.448580,-0.080262,0.630030,0.321110,-0.467650,0.227860,0.360340,-0.378180,-0.566570,0.044691,0.303920
2,.,0.151640,0.301770,-0.167630,0.176840,0.317190,0.339730,-0.434780,-0.310860,-0.449990,-0.294860,0.166080,0.119630,-0.413280,-0.423530,0.598680,0.288250,-0.115470,-0.041848,-0.679890,-0.250630,0.184720,0.086876,0.465820,0.015035,0.043474,-1.46710,-0.303840,-0.023441,0.305890,-0.217850,3.7460,0.004228,-0.184360,-0.462090,0.098329,-0.119070,0.239190,0.116100,0.417050,0.056763,-0.000064,0.068987,0.087939,-0.102850,-0.139310,0.223140,-0.080803,-0.356520,0.016413,0.102160
3,of,0.708530,0.570880,-0.471600,0.180480,0.544490,0.726030,0.181570,-0.523930,0.103810,-0.175660,0.078852,-0.362160,-0.118290,-0.833360,0.119170,-0.166050,0.061555,-0.012719,-0.566230,0.013616,0.228510,-0.143960,-0.067549,-0.381570,-0.236980,-1.70370,-0.866920,-0.267040,-0.258900,0.176700,3.8676,-0.161300,-0.132730,-0.688810,0.184440,0.005246,-0.338740,-0.078956,0.241850,0.365760,-0.347270,0.284830,0.075693,-0.062178,-0.389880,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
4,to,0.680470,-0.039263,0.301860,-0.177920,0.429620,0.032246,-0.413760,0.132280,-0.298470,-0.085253,0.171180,0.224190,-0.100460,-0.436530,0.334180,0.678460,0.057204,-0.344480,-0.427850,-0.432750,0.559630,0.100320,0.186770,-0.268540,0.037334,-2.09320,0.221710,-0.398680,0.209120,-0.557250,3.8826,0.474660,-0.956580,-0.377880,0.208690,-0.327520,0.127510,0.088359,0.163510,-0.216340,-0.094375,0.018324,0.210480,-0.030880,-0.197220,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
5,and,0.268180,0.143460,-0.278770,0.016257,0.113840,0.699230,-0.513320,-0.473680,-0.330750,-0.138340,0.270200,0.309380,-0.450120,-0.412700,-0.099320,0.038085,0.029749,0.100760,-0.250580,-0.518180,0.345580,0.449220,0.487910,-0.080866,-0.101210,-1.37770,-0.108660,-0.232010,0.012839,-0.465080,3.8463,0.313620,0.136430,-0.522440,0.330200,0.337070,-0.356010,0.324310,0.120410,0.351200,-0.069043,0.368850,0.251680,-0.245170,0.253810,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
6,in,0.330420,0.249950,-0.608740,0.109230,0.036372,0.151000,-0.550830,-0.074239,-0.092307,-0.328210,0.095980,-0.822690,-0.367170,-0.670090,0.429090,0.016496,-0.235730,0.128640,-1.095300,0.433340,0.570670,-0.103600,0.204220,0.078308,-0.427950,-1.79840,-0.278650,0.119540,-0.126890,0.031744,3.8631,-0.177860,-0.082434,-0.626980,0.264970,-0.057185,-0.073521,0.461030,0.308620,0.124980,-0.486090,-0.008027,0.031184,-0.365760,-0.426990,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
7,a,0.217050,0.465150,-0.467570,0.100820,1.013500,0.748450,-0.531040,-0.262560,0.168120,0.131820,-0.249090,-0.441850,-0.217390,0.510040,0.134480,-0.431410,-0.031230,0.206740,-0.781380,-0.201480,-0.097401,0.160880,-0.618360,-0.185040,-0.124610,-2.25260,-0.223210,0.504300,0.322570,0.153130,3.9636,-0